## Overview of Dataset, Missing Values, Data Types

In [2]:
# imports and load dataset
import pandas as pd
from datetime import datetime
from datetime import date
import numpy as np

# depending on month and year, load the respective dataset
month = '03' # example: March
year = '2009' # example: year 2009
filename = f'C:/Users/dispatching-system/Downloads/NYCWebsite_Data/unprocessed_data/yellow_tripdata_{year}-{month}.csv'
df = pd.read_csv(filename)
print(df.head())

  vendor_name Trip_Pickup_DateTime Trip_Dropoff_DateTime  Passenger_Count  \
0         CMT  2009-03-26 15:30:14   2009-03-26 15:33:45                1   
1         CMT  2009-03-07 00:09:04   2009-03-07 00:16:06                1   
2         DDS  2009-03-11 19:49:43   2009-03-11 20:00:51                1   
3         CMT  2009-03-06 22:09:53   2009-03-06 22:16:55                3   
4         CMT  2009-03-06 22:55:45   2009-03-06 23:11:35                2   

   Trip_Distance  Start_Lon  Start_Lat  Rate_Code  store_and_forward  \
0            0.3 -73.970709  40.796382        NaN                0.0   
1            1.6 -74.007315  40.739964        NaN                NaN   
2            2.0 -73.976375  40.756729        NaN                NaN   
3            1.6 -73.990625  40.751453        NaN                NaN   
4            3.8 -73.993798  40.734164        NaN                NaN   

     End_Lon    End_Lat Payment_Type  Fare_Amt  surcharge  mta_tax  Tip_Amt  \
0 -73.973602  40.792058  

In [4]:
# compare columns of datasets
i = 1
column_list = []
while i < 12:
    if len(str(i))==1:
        filename = f'C:/Users/dispatching-system/Downloads/NYCWebsite_Data/unprocessed_data/yellow_tripdata_{year}-0{i}.csv'
    else:
        filename = f'C:/Users/dispatching-system/Downloads/NYCWebsite_Data/unprocessed_data/yellow_tripdata_{year}-{i}.csv'
    df_test = pd.read_csv(filename)
    column_list.append(df_test.columns)
    i += 1

TypeError: unsupported operand type(s) for -: 'range' and 'int'

In [6]:
for counter in range(len(column_list)-1):
    if list(column_list[counter]).sort()!=list(column_list[counter+1]).sort():
        print(f'Keine Übereinstimmung: {counter}')
        break
    else:
        print("Übereinstimmung")
# normalize column names and drop some special columns for the years ...
# here code

Übereinstimmung
Übereinstimmung
Übereinstimmung
Übereinstimmung
Übereinstimmung
Übereinstimmung
Übereinstimmung
Übereinstimmung
Übereinstimmung
Übereinstimmung


In [7]:
df.dtypes

vendor_name               object
Trip_Pickup_DateTime      object
Trip_Dropoff_DateTime     object
Passenger_Count            int64
Trip_Distance            float64
Start_Lon                float64
Start_Lat                float64
Rate_Code                float64
store_and_forward        float64
End_Lon                  float64
End_Lat                  float64
Payment_Type              object
Fare_Amt                 float64
surcharge                float64
mta_tax                  float64
Tip_Amt                  float64
Tolls_Amt                float64
Total_Amt                float64
dtype: object

In [8]:
# check whether any values are null
df.isnull().sum()

vendor_name                     0
Trip_Pickup_DateTime            0
Trip_Dropoff_DateTime           0
Passenger_Count                 0
Trip_Distance                   0
Start_Lon                       0
Start_Lat                       0
Rate_Code                14387371
store_and_forward         9274078
End_Lon                         0
End_Lat                         0
Payment_Type                    0
Fare_Amt                        0
surcharge                       0
mta_tax                  14387371
Tip_Amt                         0
Tolls_Amt                       0
Total_Amt                       0
dtype: int64

## Remove irrelevant columns, transform columns

In [9]:
# define columns to drop
to_drop=['Rate_Code', 'store_and_forward', 'Payment_Type', 'surcharge','mta_tax', 'Tolls_Amt', 'Fare_Amt']
df.drop(to_drop,inplace=True,axis=1)
df.head()

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,End_Lon,End_Lat,Tip_Amt,Total_Amt
0,CMT,2009-03-26 15:30:14,2009-03-26 15:33:45,1,0.3,-73.970709,40.796382,-73.973602,40.792058,0.0,4.1
1,CMT,2009-03-07 00:09:04,2009-03-07 00:16:06,1,1.6,-74.007315,40.739964,-74.004717,40.751879,0.0,7.0
2,DDS,2009-03-11 19:49:43,2009-03-11 20:00:51,1,2.0,-73.976375,40.756729,-73.954439,40.767205,0.0,8.6
3,CMT,2009-03-06 22:09:53,2009-03-06 22:16:55,3,1.6,-73.990625,40.751453,-74.008427,40.746579,0.0,7.0
4,CMT,2009-03-06 22:55:45,2009-03-06 23:11:35,2,3.8,-73.993798,40.734164,-73.958812,40.769911,0.0,12.6


In [10]:
# get actual price for tip by: total_amt - tip_amt  [because box does not pay tips]
df['total_price']=df['Total_Amt']-df['Tip_Amt']
df.drop(columns=['Total_Amt','Tip_Amt'])

# transform distance from miles to meters
df['route_length']=df['Trip_Distance']* 1609.344
df.drop('Trip_Distance',inplace=True,axis=1)

In [11]:
# extract day, hour and minute from datetime
df['Trip_Pickup_DateTime']=pd.to_datetime(df['Trip_Pickup_DateTime'])
df['Trip_Dropoff_DateTime']=pd.to_datetime(df['Trip_Dropoff_DateTime'])

pickup_days = []
pickup_hours = []
pickup_minutes = []
for index, row in df.iterrows():
    pickup_days.append(row['Trip_Pickup_DateTime'].day)
    pickup_hours.append(row['Trip_Pickup_DateTime'].hour)
    pickup_minutes.append(row['Trip_Pickup_DateTime'].minute)
df['pickup_day']=pickup_days
df['pickup_hour']=pickup_hours
df['pickup_minute']=pickup_minutes

df.head()

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Start_Lon,Start_Lat,End_Lon,End_Lat,Tip_Amt,Total_Amt,total_price,route_length,pickup_day,pickup_hour,pickup_minute
0,CMT,2009-03-26 15:30:14,2009-03-26 15:33:45,1,-73.970709,40.796382,-73.973602,40.792058,0.0,4.1,4.1,482.8032,26,15,30
1,CMT,2009-03-07 00:09:04,2009-03-07 00:16:06,1,-74.007315,40.739964,-74.004717,40.751879,0.0,7.0,7.0,2574.9504,7,0,9
2,DDS,2009-03-11 19:49:43,2009-03-11 20:00:51,1,-73.976375,40.756729,-73.954439,40.767205,0.0,8.6,8.6,3218.6880,11,19,49
3,CMT,2009-03-06 22:09:53,2009-03-06 22:16:55,3,-73.990625,40.751453,-74.008427,40.746579,0.0,7.0,7.0,2574.9504,6,22,9
4,CMT,2009-03-06 22:55:45,2009-03-06 23:11:35,2,-73.993798,40.734164,-73.958812,40.769911,0.0,12.6,12.6,6115.5072,6,22,55


In [23]:
# rename columns to be similar to those of Kaggle
df.rename(columns={'vendor_name': 'provider_name', 'Passenger_Count': 'passenger_count', 'Trip_Pickup_DateTime': 'pickup_datetime', 'Trip_Dropoff_DateTime': 'dropoff_datetime', 'Start_Lon': 'pickup_longitude', 'Start_Lat': 'pickup_latitude', 'End_Lon': 'dropoff_longitude', 'End_Lat': 'dropoff_latitude'})

,provider_name,pickup_datetime,dropoff_datetime,Passenger_Count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,Tip_Amt,Total_Amt,total_price,route_length
0,CMT,2009-03-26 15:30:14,2009-03-26 15:33:45,1,-73.970709,40.796382,-73.973602,40.792058,0.0,4.10,4.10,482.80320
1,CMT,2009-03-07 00:09:04,2009-03-07 00:16:06,1,-74.007315,40.739964,-74.004717,40.751879,0.0,7.00,7.00,2574.95040
2,DDS,2009-03-11 19:49:43,2009-03-11 20:00:51,1,-73.976375,40.756729,-73.954439,40.767205,0.0,8.60,8.60,3218.68800
3,CMT,2009-03-06 22:09:53,2009-03-06 22:16:55,3,-73.990625,40.751453,-74.008427,40.746579,0.0,7.00,7.00,2574.95040
4,CMT,2009-03-06 22:55:45,2009-03-06 23:11:35,2,-73.993798,40.734164,-73.958812,40.769911,0.0,12.60,12.60,6115.50720
...,...,...,...,...,...,...,...,...,...,...,...,...
14387366,VTS,2009-03-30 02:29:00,2009-03-30 02:45:00,1,-74.000095,40.729247,-73.952122,40.688277,0.0,14.20,14.20,7676.57088
14387367,VTS,2009-03-30 06:48:00,2009-03-30 06:51:00,1,-73.975810,40.789140,-73.970583,40.796790,0.0,4.10,4.10,981.69984
14387368,VTS,2009-03-25 14:00:00,2009-03-25 14:11:00,1,-73.986047,40.759707,-73.968665,40.766575,0.0,7.70,7.70,2945.09952
14387369,VTS,2009-03-29 17:46:00,2009-03-29 18:22:00,1,-73.989515,40.758777,-73.788120,40.647427,5.0,54.15,49.15,27423.22176


## Remove outliers and trips that do not go through Manhattan

In [ ]:
# remove trips with passenger_count > 4 (more do normally not fit in car)
df.drop(df.index[df['passenger_count']>4],inplace=True)

# remove trips with price > 200 (even if 4 people take car and each pay 50 that seems a lot)
df.drop(df.index[df['total_price']>200],inplace=True)

In [ ]:
# define coordinate limits
# e.g., all points with latitude smaller than 40.70 and greater than 40.90 are definitely outside of Manhattan
manhattan_lat_limits = np.array([40.70, 40.90])
manhattan_lon_limits = np.array([-74.016, -73.9102])

# define new dataframe and store trips that are beyond the limits
# df_other = pd.DataFrame(colummns=df.columns.tolist())
# df_other.append(df[(df['pickup_latitude'] < manhattan_lat_limits[0]) & (df['pickup_latitude']  > manhattan_lat_limits[1])])
# df_other.append(df[(df['dropoff_latitude'] < manhattan_lat_limits[0]) & (df['dropoff_latitude']  > manhattan_lat_limits[1])])
# df_other.append(df[(df['pickup_longitude'] < manhattan_lon_limits[0]) & (df['pickup_longitude'] > manhattan_lon_limits[1])])
# df_other.append(df[(df['dropoff_longitude'] >= manhattan_lon_limits[0]) & (df['dropoff_longitude'] <= manhattan_lon_limits[1])])

# remove outlier trips from original dataframe
df = df[(df['pickup_latitude']   >= manhattan_lat_limits[0] ) & (df['pickup_latitude']   <= manhattan_lat_limits[1]) ]
df = df[(df['dropoff_latitude']  >= manhattan_lat_limits[0] ) & (df['dropoff_latitude']  <= manhattan_lat_limits[1]) ]
df = df[(df['pickup_longitude']  >= manhattan_lon_limits[0]) & (df['pickup_longitude']  <= manhattan_lon_limits[1])]
df = df[(df['dropoff_longitude'] >= manhattan_lon_limits[0]) & (df['dropoff_longitude'] <= manhattan_lon_limits[1])]


In [ ]:
# test write into csv
from pathlib import Path  
filepath = Path(f'C:/Users/dispatching-system/Downloads/NYCWebsite_Data/trips_test_{year}_{month}.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

df1 = df
df2 = pd.read_csv(f'C:/Users/dispatching-system/Downloads/Kaggle_Data/train/train.csv')
# read columns of train of Kaggle and of the test csv and compare them
if(list(df1.columns).sort()!=list(df2.columns).sort()):
    print(df1.columns)
    print(df2.columns)

In [ ]:
# out of the previously detected outlier trips: select those that go through Manhattan
# -> adapt pickup and dropoff coordinates to the respective enter and exit points in Manhattan
# -> adapt pickup and dropoff datetimes
# -> adapt route_length and total_price

In [ ]:
# todo: apply pre-processing to all months in all years (2009, 01 until 2017, 12)
# for this transfer all the pre-processing to py-file and make loop with arguments month and year?

## Other ideas (not relevant at the moment or did not work)

In [ ]:
# all data in one dataframe
# dataframes = []
# #for i in range(9):
#    i+=1
#    filename='C:/Users/dispatching-system/Downloads/yellow_tripdata_2009-0'+str(i)+'.csv'
#    file=pd.read_csv(filename)
#    file.drop(columns=to_drop)
#    dataframes.append(file)
# result = pd.concat(dataframes)